In [27]:
import pandas as pd
import numpy as np

x_all = pd.read_csv("./xevals/xeval_10_1.csv", header=None)
cas_data = pd.read_csv("sum_log.csv")
for i in range(len(cas_data)):
    print(cas_data.ix[i,1])

100-41-4
100-42-5
100-44-7
100-46-9
100-47-0
100-51-6
100-52-7
100-55-0
100-61-8
100-63-0
100-65-2
100-66-3
100-68-5
100-75-4
10004-44-1
1002-84-2
101-21-3
101-27-9
101-31-5
101-42-8
101-77-9
101-81-5
101-84-8
102-07-8
102-69-2
102-76-1
102-82-9
102170-44-5
1022-46-4
103-29-7
103-33-3
103-36-6
103-65-1
103-69-5
103-70-8
103-73-1
103-82-2
103-84-4
103-85-5
103-88-8
103-90-2
104-46-1
104-51-8
104-75-6
104-76-7
104-98-3
105-05-5
105-30-6
105-37-3
105-53-3
105-54-4
105-55-5
105-57-7
105-66-8
105-67-9
106-30-9
106-32-1
106-34-3
106-37-6
106-38-7
106-39-8
106-41-2
106-42-3
106-43-4
106-44-5
106-46-7
106-47-8
106-48-9
106-49-0
106-50-3
106-51-4
106-55-8
106-57-0
106-63-8
106-70-7
106-93-4
106-94-5
106-95-6
106-99-0
107-02-8
107-04-0
107-05-1
107-06-2
107-08-4
107-13-1
107-31-3
107-82-4
107-83-5
107-87-9
107-92-6
1077-16-3
108-08-7
108-10-1
108-11-2
108-20-3
108-21-4
108-36-1
108-37-2
108-38-3
108-39-4
108-42-9
108-43-0
108-44-1
108-46-3
108-47-4
108-48-5
108-67-8
108-68-9
108-70-3
108-85-0
10

In [19]:
sampler = np.random.permutation(len(cas_data))

In [21]:
cas_source = (cas_data.ix[:,1]).take(sampler[100:])

In [23]:
cas_source[0]

'100-41-4'

In [17]:
cas_data.ix[:,1]

0          100-41-4
1          100-42-5
2          100-44-7
3          100-46-9
4          100-47-0
5          100-51-6
6          100-52-7
7          100-55-0
8          100-61-8
9          100-63-0
10         100-65-2
11         100-66-3
12         100-68-5
13         100-75-4
14       10004-44-1
15        1002-84-2
16         101-21-3
17         101-27-9
18         101-31-5
19         101-42-8
20         101-77-9
21         101-81-5
22         101-84-8
23         102-07-8
24         102-69-2
25         102-76-1
26         102-82-9
27      102170-44-5
28        1022-46-4
29         103-29-7
           ...     
1124        96-47-9
1125        96-48-0
1126        96-50-4
1127        96-88-8
1128        97-23-4
1129        97-53-0
1130        97-59-6
1131        97-77-8
1132        97-95-0
1133       977-79-7
1134        98-01-1
1135        98-06-6
1136        98-16-8
1137        98-54-4
1138        98-82-8
1139        98-85-1
1140        98-86-2
1141        98-89-5
1142        98-92-0


In [48]:
import numpy as np
import pandas as pd
import sys
from PEIOPT_for_loop import *

x_all = pd.read_csv("x_all.csv", header=None)
y_all = pd.read_csv("y_all.csv", header=None)
cas_data = pd.read_csv("sum_log.csv")
for num_of_sample in range(50,99,50):
	for random_num in range(2):

		sampler = np.random.permutation(len(x_all))

		learn_x = x_all.take(sampler[:num_of_sample])
		learn_y = y_all.take(sampler[:num_of_sample])
		cas_source = (cas_data.ix[:,1]).take(sampler[num_of_sample:])
		test_x = x_all.take(sampler[num_of_sample:])
		test_y = y_all.take(sampler[num_of_sample:])
		cas_source = list(cas_source.values.flatten())
    

		learn_x.to_csv("./x_csvs/x_%i_%i.csv" %(num_of_sample, random_num), index=False, header=False)
		learn_y.to_csv("./y_csvs/y_%i_%i.csv" %(num_of_sample, random_num), index=False, header=False)
		test_x.to_csv("./xevals/xeval_%i_%i.csv" %(num_of_sample, random_num), index=False, header=False)
		test_y.to_csv("./yevals/yeval_%i_%i.csv" %(num_of_sample, random_num), index=False, header=False)

		
			#最適化手法本体
		
			# データロード
		X = np.loadtxt("./x_csvs/x_%i_%i.csv" %(num_of_sample, random_num),delimiter=",")
		X = X.astype(np.float64)#設計変数
		ycsv = pd.read_csv("./y_csvs/y_%i_%i.csv" %(num_of_sample, random_num), header=None)
		y = ycsv.as_matrix()
		y = y.astype(np.float64)#特性
		boundcsv = np.loadtxt("bound.csv",delimiter=",")#設計変数のレンジ
		yinfcsv = np.loadtxt("yinf.csv",delimiter=",")#各特性に対する要求
		option = pd.read_csv("option.csv", header=None)#option

		#二次元配列に統一
		if boundcsv.ndim == 1:
		    boundcsv = boundcsv[:,np.newaxis]
		    X = X[:,np.newaxis]
		if yinfcsv.ndim == 1:
		    yinfcsv = yinfcsv[:,np.newaxis]

		#boundとyinfをリストの形にする
		bounds = []
		yinf = []
		for var in range(0, boundcsv.shape[1]):
		    bounds.append([boundcsv[0,var],boundcsv[1,var]])
		for var in range(0, yinfcsv.shape[1]):
		    yinf.append([yinfcsv[0,var],yinfcsv[1,var],yinfcsv[2,var]])
		    
		optfunc_first = []
		Xselec = []
		theta = []
		result = []

		# 任意の数の候補を選択するまでループ
		for i in range(0,int(option[1][1])):
		    # GPモデルの構築
		    if i == 0: #最初は全ての特性値に対してモデルを構築する
		        gp_list = []#各特性ごとに作成したGPモデルを格納するリスト
		        for obj in range(0,len(yinf)):
		            gp = GaussianProcess(thetaL = 0.0000001, thetaU = 100)
		            Xdb = X[np.isnan(y[:,obj]) == False, :]#yが欠損値の場合は無視
		            ydb = y[np.isnan(y[:,obj]) == False, obj]#yが欠損値の場合は無視
		            
		            gp.fit(Xdb, ydb)
		            gp_list.append(gp)
		            
		    else:#二回目以降は最適化する特性のみのモデルを再構築する。
		        gp = GaussianProcess(theta0 = theta)
		        Xdb = X[np.isnan(y[:,optfunc_first.optobj]) == False, :]#yが欠損値の場合は無視
		        ydb = y[np.isnan(y[:,optfunc_first.optobj]) == False, optfunc_first.optobj]#yが欠損値の場合は無視

		        #ダミーデータをデータベースに追加
		        Xdb = np.r_[Xdb, Xselec]
		        ydb = np.r_[ydb[:,np.newaxis], optfunc_first.CL[int(option[1][2])]*np.ones([i,1])]
		        
		        gp.fit(Xdb, ydb)
		        gp_list[optfunc_first.optobj] = gp
		        
		    # DEの準備
		    optfunc = Opt_func(y = y, model = gp_list, yinfdata = yinf, option = option)#最適化する関数を含むオブジェクトを作成  
		    if i == 0:
		        #最初に作成したモデルは保存しておく
		        gp_list_first = gp_list.copy()
		        optfunc_first = Opt_func(y = y, model = gp_list_first, yinfdata = yinf, option = option)
		        #2回目以降はこのハイパーパラメータを用いてモデルを構築する
		        theta = gp_list[optfunc_first.optobj].theta_
		    
		    # DE
		    if int(option[1][4]) == 0:
		        if int(option[1][0]) == 0:
		            de_result = differential_evolution(optfunc.P, bounds)#Pで選択
		        elif int(option[1][0]) == 1:
		            de_result = differential_evolution(optfunc.EI, bounds)#EIで選択
		        elif int(option[1][0]) == 2:
		            de_result = differential_evolution(optfunc.Y, bounds)#予測値で選択
		            
		    elif int(option[1][4]) == 1:
		        #評価を行うリストを読み込み
		        Xeval = np.loadtxt("./xevals/xeval_%i_%i.csv" %(num_of_sample, random_num),delimiter=",")
		        Xeval = Xeval.astype(np.float64)
		    
		        #評価
		        if int(option[1][0]) == 0:
		            eval = optfunc.P(Xeval)#Pで選択
		        elif int(option[1][0]) == 1:
		            eval = optfunc.EI(Xeval)#EIで選択
		        elif int(option[1][0]) == 2:
		            eval = optfunc.Y(Xeval)#予測値で選択
		            
		        #候補の選択
		        de_result = result_cont(Xeval[eval.argmin(),:],eval.argmin())
		        
		    #選択された候補を格納
		    if i == 0:
		        Xselec = de_result.x[:,np.newaxis].T
		    else:
		        Xselec = np.r_[Xselec, de_result.x[:,np.newaxis].T]
		        
		    #出力用のデータフレームを作成
		    result_temp = pd.DataFrame(de_result.x.T).T#Xを格納
		    result_temp[result_temp.shape[1]] = de_result.message#DEの最適化の終了メッセージを格納
		    
		    #最初に作成したモデル（ダミーデータを含まない）で評価を行い、その評価値を格納する
		    if int(option[1][0]) == 0:
		        eve = optfunc_first.P(de_result.x[:,np.newaxis].T)
		    elif int(option[1][0]) == 1:
		        eve = optfunc_first.EI(de_result.x[:,np.newaxis].T)
		        result_of_y_pred = optfunc_first.ypred
		    elif int(option[1][0]) == 2:
		        eve = optfunc_first.Y(de_result.x[:,np.newaxis].T)
		        if optfunc_first.optdirection < 0:
		            eve = -eve
		        
		    result_temp[result_temp.shape[1]] = -eve
		    result_temp[result_temp.shape[1]] = "id"
		    result_temp[result_temp.shape[1]] = '-' if (int(option[1][4]) == 0) else de_result.id

		    print(result_of_y_pred[0])
		    
		    #出力用のデータフレームを更新
		    if i == 0:
		        result = result_temp
		    else:
		        result = pd.concat([result,result_temp], axis=0)
		y_pred_value = pd.Series(result_of_y_pred[0])
		result = pd.concat([result,y_pred_value], axis=1)
		print("succeed")
		print(de_result.id)
		print(cas_source[de_result.id])
		print(len(test_x))

[-9.22876698]
succeed
226
33284-53-6
1104
[-6.35965751]
succeed
742
93-76-5
1104


In [ ]:
y_eval = pd.read_csv(".csv", header=None)

In [51]:
y_eval = pd.read_csv("yeval.csv",header=None)

In [52]:
y_eval

,0
0,-6.0
1,-4.5
2,-7.8
3,-5.7
4,-4.9
5,-5.4
6,-4.4
7,-5.8
8,-5.7
9,-6.8


In [2]:
x_all

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,1,0,0,260,10,10,0,0,140.0500,0,...,0,0,0,0,5,4,0,0,0,5
1,0,0,0,177,8,7,2,2,106.0590,0,...,0,0,0,3,2,1,1,0,0,8
2,1,0,1,96,6,6,6,0,72.0642,0,...,0,0,0,0,0,0,0,0,1,1
3,2,0,2,584,20,21,12,1,427.7520,0,...,0,0,1,8,8,0,0,0,2,10
4,4,0,4,288,18,21,18,0,216.1930,0,...,0,0,8,0,0,0,0,0,4,1
5,0,0,0,264,14,13,2,10,176.1270,0,...,0,0,0,10,2,0,0,0,0,14
6,1,1,0,206,8,8,6,0,112.0440,0,...,0,0,0,0,3,2,0,0,0,4
7,2,0,2,205,11,12,10,0,138.1020,0,...,0,0,0,0,1,1,0,0,2,2
8,3,0,3,260,15,17,14,0,184.1490,0,...,0,0,2,0,1,1,0,0,3,2
9,2,0,1,305,16,17,10,3,202.1450,0,...,0,0,0,4,2,1,0,0,1,6


In [3]:
len(range(10,51,10))

5

In [4]:
len(range(20))

20